In [1]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [29]:
import math
import copy
import cv2
import numpy as np
import random
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from kaggle_problems.rosneft_proppant.helpers import *

In [25]:
DATA_DIR = "kaggle_problems/rosneft_proppant/data"
ORIGINAL_IMG_DIR = "kaggle_problems/rosneft_proppant/data/train"
DEBUG_IMG_DIR = "kaggle_problems/rosneft_proppant/data/debug"

In [396]:
# def almost_equal(a, b, e):
#     return abs(a - b) < e

# def get_circle_diff(contour):
#     s = cv2.contourArea(contour) # pi * R ^ 2
#     p = cv2.arcLength(contour, True) # 2 * pi * R
    
#     s /= math.pi
#     p /= math.pi * 2
#     if (s + p * p < 1e-5):
#         return 1. # MAX
    
#     return abs(s - p * p) / (s + p * p)

# def is_circle(contour):
#     s = cv2.contourArea(contour)
#     if (s > 20):
#         return get_circle_diff(contour) < 0.1
#     else:
#         return False


# def get_almost_convex(contour):
#     convex = cv2.convexHull(contour)
#     origin_area = cv2.contourArea(contour)
#     conveex_area = cv2.contourArea(convex)
#     return origin_area / max(1, conveex_area)

# def is_almost_convex(contour):
#     return get_almost_convex(contour) >= 0.9
    

In [412]:
# def get_contour_color(img, contour):
#     if (len(contour) == 0):
#         return None
#     return img[contour[0][0][1], contour[0][0][0]]

# def get_contour_center(contour):
#     if (len(contour) == 0):
#         return None
#     x, y = 0, 0
#     for sell in contour:
#         x += sell[0][1]
#         y += sell[0][0]
#     return (y / len(contour), x / len(contour))


In [12]:
def process_bw_img(original_img, img_number):
    result_circles = []
    os.mkdir("{}/{}".format(DEBUG_IMG_DIR, img_number)) # create debug dir
    
    img_with_circle = copy.deepcopy(img)
    img = copy.deepcopy(original_img)

    gaussian_3 = cv2.GaussianBlur(img, (0, 0), 2.0)
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    cv2.imwrite("{}/img.jpg".format(DEBUG_IMG_DIR), gray)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    th, threshed = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    it = 0
    for r in range(15, 2, -1):
        print('-' * 30)
        # Выделение кругов
        circles = cv2.HoughCircles(threshed,cv2.HOUGH_GRADIENT,0.5,minDist=r,
                                param1=200,param2=1,minRadius=r,maxRadius=r)
        circles = circles[0]
        print("Circle count: {}. r: {}".format(len(circles), r))
        filtered_circle = [(int(circle[0]), int(circle[1]), int(circle[2])) for circle in circles]

        filtered_circle = [circle for circle in filtered_circle \
        if is_convex_in_circle(threshed, circle[0], circle[1], circle[2])]

        print("After first filter: {}. r: {}".format(len(filtered_circle), r))

        filtered_circle = [circle for circle in filtered_circle \
            if is_border_white_circle(threshed, circle[0], circle[1], circle[2])]

        print("After second filter: {}. r: {}".format(len(filtered_circle), r))

        for circle in filtered_circle:
            cv2.circle(threshed,(circle[0],circle[1]),circle[2], 0, -1) # fill with 0 because of THRESH_BINARY_INV
            cv2.circle(original_img,(circle[0],circle[1]),circle[2],(255, 255, 255),-1)
            cv2.circle(img_with_circle,(circle[0],circle[1]),circle[2], get_random_color(),1)

        cv2.imwrite("{}/{}/original_img_{}.jpg".format(DEBUG_IMG_DIR, img_number, it), original_img)
        cv2.imwrite("{}/{}/img_with_circle_{}.jpg".format(DEBUG_IMG_DIR, img_number, it), img_with_circle)
        cv2.imwrite("{}/{}/threshed_{}.jpg".format(DEBUG_IMG_DIR, img_number, it), threshed)
        it += 1
        result_circles.extend(filtered_circle)
        
    return result_circles

In [30]:
all_img = os.listdir(ORIGINAL_IMG_DIR)

for img_name in all_img:
    img_number, _ = img_name.split('.')
    original_img = cv2.imread("{}/{}".format(ORIGINAL_IMG_DIR, img_name)) 
    Path("{}/processing/bw_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    Path("{}/processing/color_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    
    if is_grey_img(original_img):
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/bw_images/{}".format(DATA_DIR, img_name))
    else:
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/color_images/{}".format(DATA_DIR, img_name))


TypeError: 'NoneType' object is not subscriptable

In [19]:
a = [1, 2]
b, c = a